In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

In [2]:
path = "../s1/data/"
file_name = "Online Retail.xlsx"
sheet_name = "Online Retail"
df = pd.read_excel(path + file_name, sheet_name=sheet_name)
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [10]:
df = df[df.notna()]
X = df[df["Quantity"] > 0][["CustomerID", "StockCode", "InvoiceDate", "Country", "UnitPrice", "Quantity"]]
ids = df["StockCode"]
counts = pd.DataFrame(df["StockCode"].value_counts())
# X.insert(0, 'Reorder', 0)
# X.loc[ids.isin(ids[ids.duplicated()]), "Reorder"] = 1
X = pd.merge(X, counts, right_index=True, left_on="StockCode").drop(["StockCode_x"], axis=1).rename(columns={'StockCode_y':'OrderCount'})

groups = X.groupby('CustomerID', as_index=False)["StockCode"].value_counts()
X = pd.merge(X, groups, on=["CustomerID", "StockCode"])

X

,StockCode,CustomerID,InvoiceDate,Country,UnitPrice,Quantity,OrderCount,count
0,85123A,17850.0,2010-12-01 08:26:00,United Kingdom,2.55,6,2313,17
1,85123A,17850.0,2010-12-01 09:02:00,United Kingdom,2.55,6,2313,17
2,85123A,17850.0,2010-12-01 09:32:00,United Kingdom,2.55,6,2313,17
3,85123A,17850.0,2010-12-01 10:51:00,United Kingdom,2.55,6,2313,17
4,85123A,17850.0,2010-12-01 11:33:00,United Kingdom,2.55,8,2313,17
...,...,...,...,...,...,...,...,...
397919,23561,14646.0,2011-12-07 15:16:00,Netherlands,2.49,60,16,1
397920,23561,12621.0,2011-12-08 11:25:00,Germany,2.89,12,16,1
397921,23561,17581.0,2011-12-09 12:20:00,United Kingdom,2.89,6,16,1
397922,90214U,13790.0,2011-12-05 15:48:00,United Kingdom,0.29,12,1,1


In [11]:
X['Time'] = pd.to_datetime(X['InvoiceDate']).dt.hour
X["DayOfTheWeek"] = X["InvoiceDate"].dt.day_name()

times = pd.get_dummies(X["Time"], dummy_na=False)
timesColumns={i: "Hour " + str(i) for i in range(6,21)}#6-20
times = times.rename(columns=timesColumns)
X = X.drop('Time', axis = 1, errors='ignore')
# Join the encoded df
X = X.join(times)

days = pd.get_dummies(X["DayOfTheWeek"], dummy_na=False)
X = X.drop('DayOfTheWeek', axis = 1, errors='ignore')
# Join the encoded df
X = X.join(days)

X = X.drop(["InvoiceDate"], axis=1)

countries = pd.get_dummies(X["Country"], dummy_na=False)
X = X.drop('Country', axis = 1, errors='ignore')
# Join the encoded df
X = X.join(countries)
X = X[X["Unspecified"]==0]
X = X.drop(["Unspecified"], axis=1)

# customers = pd.get_dummies(X["CustomerID"], dummy_na=False)
X = X.drop('CustomerID', axis = 1, errors='ignore')
# Join the encoded df
# X = X.join(customers)

le_product = LabelEncoder()
products = le_product.fit_transform(X["StockCode"].apply(str))
# products = pd.get_dummies(X["StockCode"], dummy_na=False)
X = X.drop('StockCode', axis = 1, errors='ignore')
# # Join the encoded df
X = X.join(pd.DataFrame(products, columns=["Product"]))

le_product_mapping = dict(zip(le_product.transform(le_product.classes_), le_product.classes_))

X

,UnitPrice,Quantity,OrderCount,count,Hour 6,Hour 7,Hour 8,Hour 9,Hour 10,Hour 11,...,RSA,Saudi Arabia,Singapore,Spain,Sweden,Switzerland,USA,United Arab Emirates,United Kingdom,Product
0,2.55,6,2313,17,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,3233.0
1,2.55,6,2313,17,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,3233.0
2,2.55,6,2313,17,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,3233.0
3,2.55,6,2313,17,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,3233.0
4,2.55,8,2313,17,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,3233.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397919,2.49,60,16,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
397920,2.89,12,16,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,NaN
397921,2.89,6,16,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
397922,0.29,12,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN


In [12]:
scaled = StandardScaler().fit(X[["UnitPrice", "Quantity", "OrderCount", "count"]]).transform(X[["UnitPrice", "Quantity", "OrderCount", "count"]])
scaled

array([[-2.56124838e-02, -3.89267714e-02,  4.57802192e+00,
         3.58577168e+00],
       [-2.56124838e-02, -3.89267714e-02,  4.57802192e+00,
         3.58577168e+00],
       [-2.56124838e-02, -3.89267714e-02,  4.57802192e+00,
         3.58577168e+00],
       ...,
       [-1.02302233e-02, -3.89267714e-02, -1.07558500e+00,
        -4.50885971e-01],
       [-1.27859275e-01, -5.68119439e-03, -1.11250451e+00,
        -4.50885971e-01],
       [-4.68761970e-02,  4.48715413e+02, -1.11004321e+00,
        -4.50885971e-01]])

In [13]:
pca = PCA(n_components=3)
X_pca = pca.fit_transform(scaled)
X_pca

array([[ 5.74084018e+00,  1.40340545e-01, -6.01566655e-01],
       [ 5.74084018e+00,  1.40340545e-01, -6.01566655e-01],
       [ 5.74084018e+00,  1.40340545e-01, -6.01566655e-01],
       ...,
       [-1.07633192e+00, -5.20843869e-02,  7.88393132e-02],
       [-1.11099834e+00,  5.05400395e-02,  1.74502684e-02],
       [ 1.83756992e+01,  3.33114533e+02,  3.00015644e+02]])

In [14]:
X = pd.concat([X.drop(["UnitPrice", "Quantity", "OrderCount", "count"], axis=1), pd.DataFrame(X_pca, columns=["Component1", "Component2", "Component3"])], axis=1, join="inner")
X

,Hour 6,Hour 7,Hour 8,Hour 9,Hour 10,Hour 11,Hour 12,Hour 13,Hour 14,Hour 15,...,Spain,Sweden,Switzerland,USA,United Arab Emirates,United Kingdom,Product,Component1,Component2,Component3
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,3233.0,5.740840,0.140341,-0.601567
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,3233.0,5.740840,0.140341,-0.601567
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,3233.0,5.740840,0.140341,-0.601567
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,3233.0,5.740840,0.140341,-0.601567
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,3233.0,5.741321,0.148567,-0.594159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397675,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,2367.0,-1.064907,0.182156,0.265495
397676,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,2367.0,-1.074889,-0.027404,0.101062
397677,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,2367.0,-1.076332,-0.052084,0.078839
397678,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,3654.0,-1.110998,0.050540,0.017450


In [15]:
X.to_csv("preprocessed.xlsx", index=False)

In [7]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import pandas as pd
X = pd.read_csv("preprocessed.xlsx")
X

,Hour 6,Hour 7,Hour 8,Hour 9,Hour 10,Hour 11,Hour 12,Hour 13,Hour 14,Hour 15,...,Spain,Sweden,Switzerland,USA,United Arab Emirates,United Kingdom,Product,Component1,Component2,Component3
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,3233.0,5.740840,0.140341,-0.601567
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,3233.0,5.740840,0.140341,-0.601567
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,3233.0,5.740840,0.140341,-0.601567
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,3233.0,5.740840,0.140341,-0.601567
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,3233.0,5.741321,0.148567,-0.594159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397431,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,2367.0,-1.064907,0.182156,0.265495
397432,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,2367.0,-1.074889,-0.027404,0.101062
397433,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,2367.0,-1.076332,-0.052084,0.078839
397434,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,3654.0,-1.110998,0.050540,0.017450


In [20]:
# a,b,c,d = train_test_split()
X_train, X_test, y_train, y_test = train_test_split(X.drop(["Product"], axis=1), X["Product"], train_size=0.8, test_size=0.2, random_state=0)
X_train.shape, X_test.shape

((317948, 60), (79488, 60))

In [17]:
# clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3).fit(X_train, y_train)
# clf = LogisticRegression().fit(X_train, y_train)
# clf = RandomForestClassifier().fit(X_train, y_train)
clf = DecisionTreeClassifier().fit(X_train, y_train)

In [21]:
y_pred = clf.predict(X_test)

In [22]:
score = accuracy_score(y_test, y_pred)
score

0.9230324074074074